In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
from bs4 import BeautifulSoup
import time # sleep()

In [2]:
# 1건의 대화 정보를 저장하는 클래스
class Conversation:
    def __init__(self, question, answer):
        self.question = question
        self.answer = answer
    def __str__(self):
        return '질문: {}\n답변: {}'.format(self.question, self.answer)

In [3]:
c = Conversation('너... 누구야??', '알아서 뭐하시려고...')
print(c)

질문: 너... 누구야??
답변: 알아서 뭐하시려고...


In [4]:
# 75개의 대화 주제와 세부 대화 내용의 url을 크롤링해서 리턴하는 함수
def getSubject():
    targetSite = 'https://basicenglishspeaking.com/daily-english-conversation-topics/'
    request = requests.get(targetSite)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    
    subjects = []    # 대화 주제를 저장할 빈 리스트
    contentLink = [] # 대화 주제에 따른 세부 대화 내용 페이지의 url을 저장할 빈 리스트
    divs = soup.findAll('div', {'class': 'tcb-col'})
    for div in divs:
        chapter = div.findAll('a')
        for a in chapter:
            subjects.append(a.text)
            contentLink.append(a.get('href')) # 세부 대화 내용의 url을 저장한다.
    
    # 대화 주제와 대화 주제에 따른 세부 대화 내용의 url을 리턴시킨다.
    # 리턴되는 데이터는 2개가 아니라 subjects, contentLink를 묶어서 튜플 1개로 리턴시킨다.
    return subjects, contentLink

In [5]:
subjects, contentLink = getSubject()
for i in range(len(subjects)):
    print('{0:2d}. {1} - {2}'.format(i + 1, subjects[i], contentLink[i]))

 1. Family - https://basicenglishspeaking.com/family/
 2. Restaurant - https://basicenglishspeaking.com/restaurant/
 3. Books - https://basicenglishspeaking.com/books/
 4. Travel - https://basicenglishspeaking.com/travel/
 5. Website - https://basicenglishspeaking.com/website/
 6. Accident - https://basicenglishspeaking.com/accident/
 7. Childhood memory - https://basicenglishspeaking.com/childhood-memory/
 8. Favorite rooms - https://basicenglishspeaking.com/favorite-rooms/
 9. Presents - https://basicenglishspeaking.com/presents/
10. Historical place - https://basicenglishspeaking.com/historical-place/
11. Newspaper/ Magazine - https://basicenglishspeaking.com/newspaper-magazine/
12. A memorable event - https://basicenglishspeaking.com/an-event/
13. A favorite subject - https://basicenglishspeaking.com/a-subject/
14. A museum - https://basicenglishspeaking.com/a-museum/
15. A favorite movie - https://basicenglishspeaking.com/a-movie/
16. A foreign country - https://basicenglishspeaki

In [12]:
# 대화 주제에 따른 모든 대화 내용을 저장할 빈 리스트를 선언한다.
conversation = [] # Conversation 클래스 객체를 저장할 리스트, Conversation 클래스 => 질문과 답변이 한 쌍으로 저장된 클래스

i = 0 # 대화 주제의 개수를 카운트 하는 변수
# 대화 주제의 개수만큼 반복하며 각각의 대화 주제에 따른 대화 내용을 크롤링한다.
for s in subjects[:1]:
    conversation.append('{0:2d}. {1} - {2}'.format(i + 1, subjects[i], contentLink[i]))
    
    # 대화 주제별 대화 내용을 크롤링 할 페이지를 요청한다.
    request = requests.get(contentLink[i])
    
    # time 모듈을 import 하면 sleep() 메소드를 사용할 수 있다.
    # sleep() 메소드는 인수로 지정한 시간만큼 프로그램을 멈춘다. => 시간은 초 단위로 지정한다.
    # 대형 포털 사이트는 짧은 간격으로 많은 요청이 들어오면 자기네 사이트가 공격당하는 것으로 판단할 수 있기 때문에 일정한
    # 시간 간격을 두고 정보를 요청하는 것이 좋다.
    # 페이지가 로딩되는 시간만큼 기다리기 위해서 일정 시간 간격을 두고 정보를 요청하기도 한다.
    time.sleep(1)
    
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 대화 주제에 따른 세부 대화 내용은 class 속성이 'sc_player_container1'인 div 태그의 다음 형제이다.
    divs = soup.findAll('div', {'class': 'sc_player_container1'}) # 플레이 버튼을 찾는다.
    # print(len(divs))
    
    # 대화 주제에 따른 세부 대화 내용의 개수 만큼 반복한다.
    for div in divs:
        # print(div)
        # index(): 인수로 지정된 객체의 index 번호를 얻어온다.
        # divs.index(div)
        # print(divs.index(div))
        # index() 메소드의 실행 결과가 짝수면 질문이고 홀수면 답변이다.
        
        # 크롤링할 데이터가 태그 내부에 작성된 것이 아니고 class 속성이 'sc_player_container1'인 div 태그의
        # 다음 형제로 작성되어 있다.
        # next_sibling     => 다음 형제
        # previous_sibling => 이전 형제
        if divs.index(div) % 2 == 0: # 질문인감?
            question = div.next_sibling
        else:
            answer = div.next_sibling
            
            # 질문과 답변이 한 쌍이 되었으므로 Conversation 클래스 객체를 생성하고 질문과 답변을 저장한다.
            c = Conversation(question, answer)
            # 질문과 답변이 한 쌍으로 저장된 Conversation 클래스 객체를 대화 내용을 기억하는 conversation 리스트에
            # 저장한다.
            conversation.append(c)
            
    # 대화 주제 1건의 세부 대화 내용이 크롤링되면 다음 대화를 처리하기 뮈해 i를 1증가 시킨다.
    i += 1

In [13]:
for c in conversation:
    print(c)

 1. Family - https://basicenglishspeaking.com/family/
질문: How many people are there in your family?
답변: There are 5 people in my family: my father, mother, brother, sister, and me.
질문: Does your family live in a house or an apartment?
답변: We live in a house in the countryside.
질문: What does your father do?
답변: My father is a doctor. He works at the local hospital.
질문: How old is your mother?
답변: She is 40 years old, 1 year younger than my father.
질문: Do you have any siblings? What’s his/her name?
답변: Yes, I do. I have 1 elder brother, David, and 1 younger sister, Mary.
질문: Are you the oldest amongst your brothers and sisters?
답변: No, I’m not. I’m the second child in my family.
질문: What does your mother/father like?
답변: My father likes playing football and my mother likes cooking.
질문: Do your parents let you stay out late?
답변: Of course not. They always ask me to get home before 10 pm each night.
질문: Do you stay with your parents?
답변: Right now, no, but I used to.
질문: Does your family u